In [1]:
from os.path import join, exists
from os import makedirs
import pandas as pd
import numpy as np

from brainpipe.system import study
from utils import subjects, pos_cirles, pos_up_down, pos_RL
from codes import num_odor_to_ref
from compute_odor_distances_all_pairs import compute_dist_trials_su_E, reorder_df

### create dist table for all selected trials by patient

In [ ]:
st = study('Ripples')
#any freq will do just to get the trials selection by subj
PATH = join(st.path, 'feature/theta_power_E_odors/')
filename = join(PATH, '{}_odors=all_elecs=psd_freq=h_theta_pow.npz')
PATH_SAVE = join(st.path, 'feature/distance_table_features/')
dfname = join(PATH_SAVE, 'df_trials_dims=all_odors=all_su={}.csv')
if not exists(PATH_SAVE):
    makedirs(PATH_SAVE)

PATH_b = '/media/1_Analyses_Intra_EM_Odor/1bis_OE_BaseSam/JPlailly201306_seeg_ALS/behavior/'
df_perf = join(PATH_b,'encoding_individual_results.xls')
df_od_name = join(PATH_b, 'Recap_Odeurs_Evaluations.xlsx')
cols_s = ['trial_time','odor_num','encoding_day','image_context',
          'insp_V','exp_V']
cols_od = ['od_num','Pleasantness','Familiarity','strength',
           'structure','type']

for su in subjects:
    print('processing',su)
    pow_mat = np.load(filename.format(su), allow_pickle=True)
    trials_taken = pow_mat['trials_sel'] #select trials
    od_order = np.array([int(x) for x in pow_mat['odor']])

    df_b = pd.read_excel(df_perf, sheet_name=su)[trials_taken]
    df_order = reorder_df(df_b, od_order)
    df_order['insp_V'] = np.abs(df_order['total_encodage_0_insp_volume'])
    df_order['insp_V'] = df_order['insp_V'].fillna(df_order['insp_V'].mean())
    df_order['exp_V'] = df_order['total_encodage_0_exp_volume']
    df_order['exp_V'] = df_order['exp_V'].fillna(df_order['exp_V'].mean())
    df_order = df_order[cols_s]
    
    df_od = pd.read_excel(df_od_name, sheet_name='Final')[cols_od]
    df_od['odor_num'] = [int(x[1:]) for x in df_od['od_num']]
    
    n_trials = df_order.shape[0]
    df_b_od = pd.DataFrame()
    for trial in range(n_trials):
        od = df_order['odor_num'].iloc[trial]
        od_info = df_od.loc[df_od['odor_num']==od]
        df_b_od = df_b_od.append(od_info,ignore_index=True)
    df_b_od = df_b_od.drop(['od_num'],axis=1)
    
    df_tot = pd.merge(df_order,df_od,on='odor_num')
    df_tot.to_csv(dfname.format(su,norm),index=False)

### create MCA df (categorial dimension) by subj

In [ ]:
import matplotlib.pyplot as plt
from utils import pos_cirles

for p in pos_cirles:
    plt.plot(pos_cirles[p][0],pos_cirles[p][1],'+b')
    plt.annotate(p, # this is the text
                 (pos_cirles[p]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center')
plt.ylim(0,700)
plt.xlim(-50,900)
plt.gca().invert_yaxis()
plt.show()

In [4]:
st = study('Ripples')
PATH = join(st.path, 'feature/distance_table_features/')
dfname = join(PATH, 'df_trials_dims=all_odors=all_su={}.csv')
df_save = join(PATH, 'df_dist=MCA_dims=2extr_odors=all_su={}.csv')
df_save_all = join(PATH, 'df_dist=MCA_dims=2extr_odors=all_su=ALL.csv')

df_all_su = pd.DataFrame()
for su in subjects:
    print('processing',su)
    df_d = pd.read_csv(dfname.format(su))
    df_d['Day'] = ['D'+str(x) for x in df_d['encoding_day']]
    df_d['Context'] = df_d['image_context']
    df_d['Loc'] = ['P'+str(num_odor_to_ref[su][od][1]) for od in df_d['odor_num']]
    df_d['up_down'] = [pos_up_down[p] for p in df_d['Loc']]
    df_d['R_L'] = [pos_RL[p] for p in df_d['Loc']]
    #df_d['insp_V'] = pd.qcut(df_d['insp_V'], 2, 
    #                           labels=["insp_L", "insp_H"])
    #df_d['exp_V'] = pd.qcut(df_d['exp_V'], 2, 
    #                          labels=["exp_L", "exp_H"])
    df_d['Pleasantness'] = pd.qcut(df_d['Pleasantness'], 2,
                                labels=["pleas_L", "pleas_H"])
    df_d['Familiarity'] = pd.qcut(df_d['Familiarity'], 2,
                                labels=["fam_L", "fam_H"])
    #df_d['Time_E'] = pd.qcut(df_d['trial_time'], 2,
    #                            labels=["EncT_L", "EncT_H"])
    #df_d['Time_od'] = df_d.groupby('od_num')[['trial_time']].transform(lambda x: pd.qcut(x,\
    #                                         2,labels=["OdT_L", "OdT_H"]))
    df_d = df_d.drop(['odor_num','od_num','trial_time','insp_V','exp_V',
                     'encoding_day', 'image_context','Loc'],axis=1)
    df_d['subjects'] = [su]*df_d.shape[0]
    df_d.to_csv(df_save.format(su),index=False)
    df_all_su = df_all_su.append(df_d)
df_all_su.to_csv(df_save_all,index=False)

-> Ripples loaded
processing CHAF
processing LEFC
processing FERJ
processing SEMC
processing VACJ
processing PIRJ
